In [13]:
import pandas as pd
import datetime as dt

In [99]:
validation_file = '../material_preparation_step/FlightManifest.csv'
id_file = '../step_2/id_info.json'
board_pass_file = '../step_2/pass_info.json'
baggage_file = '../step_4/lighters.json'

In [108]:
validation = pd.read_csv(validation_file)

In [109]:
validation

,Passanger Name,Carrier,Flight No.,Class,From,To,Date,Baggage,Seat,Gate,...,Ticket No.,First Name,Last Name,Date of Birth,Sex,NameValidation,DoBValidation,PersonValidation,BoardingPassValidation,LuggageValidation
0,Sameer Kumar,UA,234,Economy,San Francisco,Chicago,"April 20, 2022",YES,34A,G1,...,34236746,Sameer,Kumar,25.01.90,M,False,False,False,False,False
1,Radha S. Kumar,UA,234,Economy,San Francisco,Chicago,"April 20, 2022",YES,34B,G1,...,34236747,Radha,S. Kumar,05.03.94,F,False,False,False,False,False
2,James Webb,UA,234,Business,San Francisco,Chicago,"April 20, 2022",No,1A,G1,...,34236748,James,Webb,15.12.70,M,False,False,False,False,False
3,Libby Herold,UA,234,Business,San Francisco,Chicago,"April 20, 2022",No,3D,G1,...,34236749,Libby,Herold,10.01.96,F,False,False,False,False,False
4,James Jackson,UA,234,Economy,San Francisco,Chicago,"April 20, 2022",YES,25B,G1,...,34236750,James,Jackson,12.10.56,M,False,False,False,False,False


In [36]:
id_dataset = pd.read_json(id_file)
bp_dataset = pd.read_json(board_pass_file)

In [67]:
def validate_boarding_pass(passenger, board_pass_dataset):
    name_valid = False
    bp_validation = False
    for i, bpass in board_pass_dataset.iterrows():
        if bpass['Name'].replace('.', '') == passenger['Passanger Name'].replace('.', ''):
            print('Found boarding pass')
            name_valid = True
            if bpass['Carrier'].strip() != passenger['Carrier '].strip():
                print('Carrier not matching')
                break
            if str(bpass['Flight']).strip() != str(passenger['Flight No.']).strip():
                print('Flight number not matching')
                break
            if pd.to_datetime(bpass['Date']) != pd.to_datetime(passenger['Date']):
                print('Date not matching')
                break
            if bpass['Seat'].strip() != passenger['Seat'].strip():
                print('Seat not matching')
                break
            if bpass['Class'].strip() != passenger['Class'].strip()[0]:
                print('Class not matching')
                break
            bp_validation = True
            break
    return name_valid, bp_validation

In [68]:
def validate_face(row):
    # Not possible to implement due to API restrictions
    return False


In [69]:
def validate_baggage(image_name, predictions_data):
    # prediction generation is defined in step 4
    if image_name not in predictions_data:
        return False
    object_list = predictions_data[image_name]
    for obj in object_list:
        if list(obj.keys())[0] == 'lighter' and obj['lighter'] > 0.75:
            return True
    return False


In [118]:
def validate_id(passenger, id_dataset):
    name_validation = False
    dop_validation = False
    for i, row in id_dataset.iterrows():
        if row['FirstName'] == passenger['First Name'] and row['LastName'] == passenger['Last Name']:
            print('Found ID')
            name_validation = True
            dop_validation = pd.to_datetime(row['DateOfBirth']) == pd.to_datetime(passenger['Date of Birth'])
            break
    return (name_validation, dop_validation)




In [129]:
for i, row in validation.iterrows():
    print(f'Validating passenger {row["Passanger Name"]}')
    name_id_valid, dop_valid = validate_id(row, id_dataset)
    name_bp_valid, bp_valid = validate_boarding_pass(row, bp_dataset)
    face_valid = validate_face(row)
    baggage_valid = validate_baggage("test_image", {})
    msg = None
    if name_id_valid and name_bp_valid:
        validation.loc[i, 'NameValidation'] = True
        if bp_valid:
            msg = f"""Dear {"Mr." if row['Sex'] == 'M' else "Ms."} {row['Passanger Name']},
You are welcome to flight # {row['Flight No.']} leaving at {row["Boarding Time"]} from {row.From} to {row.To}.
Your seat number is {row['Seat']}, and it is confirmed."""
        if baggage_valid:
            msg += '\nWe did not find a prohibited item (lighter) in your carry-on baggage, thanks for following the procedure.'
        else:
            msg += '\nWe have found a prohibited item in your carry-on baggage, and it is flagged for removal.'
        if face_valid:
            msg += '\nYour identity is verified so please board the plane. '

    if not (name_id_valid and dop_valid):
        msg = """Dear Sir/Madam,
Some of the information on your ID card does not match the flight manifest data, so you cannot board the plane.
Please see a customer service representative."""

    if not bp_valid:
        msg = """Dear Sir/Madam,
Some of the information in your boarding pass does not match the flight manifest data, so you cannot board the plane.
Please see a customer service representative."""

    validation.loc[i, 'DoBValidation'] = dop_valid
    validation.loc[i, 'BoardingPassValidation'] = bp_valid
    validation.loc[i, 'PersonValidation'] = face_valid
    validation.loc[i, 'BaggageValidation'] = baggage_valid
    if msg:
        print(msg + '\n\n')


Validating passenger Sameer Kumar
Found ID
Found boarding pass
Dear Mr. Sameer Kumar,
You are welcome to flight # 234 leaving at 10:00 AM PST from San Francisco to Chicago.
Your seat number is 34A, and it is confirmed.
We have found a prohibited item in your carry-on baggage, and it is flagged for removal.


Validating passenger Radha S. Kumar
Found boarding pass
Dear Sir/Madam,
Some of the information on your ID card does not match the flight manifest data, so you cannot board the plane.
Please see a customer service representative.


Validating passenger James Webb
Found ID
Found boarding pass
Dear Mr. James Webb,
You are welcome to flight # 234 leaving at 10:00 AM PST from San Francisco to Chicago.
Your seat number is 1A, and it is confirmed.
We have found a prohibited item in your carry-on baggage, and it is flagged for removal.


Validating passenger Libby Herold
Found ID
Found boarding pass
Dear Ms. Libby Herold,
You are welcome to flight # 234 leaving at 10:00 AM PST from San Fr

In [125]:
validation

,Passanger Name,Carrier,Flight No.,Class,From,To,Date,Baggage,Seat,Gate,...,First Name,Last Name,Date of Birth,Sex,NameValidation,DoBValidation,PersonValidation,BoardingPassValidation,LuggageValidation,BaggageValidation
0,Sameer Kumar,UA,234,Economy,San Francisco,Chicago,"April 20, 2022",YES,34A,G1,...,Sameer,Kumar,25.01.90,M,True,True,False,True,False,False
1,Radha S. Kumar,UA,234,Economy,San Francisco,Chicago,"April 20, 2022",YES,34B,G1,...,Radha,S. Kumar,05.03.94,F,False,False,False,True,False,False
2,James Webb,UA,234,Business,San Francisco,Chicago,"April 20, 2022",No,1A,G1,...,James,Webb,15.12.70,M,True,True,False,True,False,False
3,Libby Herold,UA,234,Business,San Francisco,Chicago,"April 20, 2022",No,3D,G1,...,Libby,Herold,10.01.96,F,True,True,False,True,False,False
4,James Jackson,UA,234,Economy,San Francisco,Chicago,"April 20, 2022",YES,25B,G1,...,James,Jackson,12.10.56,M,True,True,False,True,False,False
